### Import Libraries

In [ ]:
# Import the required packages and libraries
import datetime
import os
from dotenv import load_dotenv 
from pathlib import Path

### Import Scripts

In [ ]:
from pyprediktormapclient.opc_ua import OPC_UA
from pyprediktormapclient.model_index import ModelIndex
from pyprediktormapclient.auth_client import AUTH_CLIENT
from pyprediktormapclient.analytics_helper import AnalyticsHelper
from pyprediktormapclient.shared import *

### Import Envrionment Variables

In [ ]:
# Consider obtaining the envrionment variables from .env file if you are running this locally from source.
dotenv_path = Path("../.env")
load_dotenv(dotenv_path=dotenv_path)

In [ ]:
username = os.environ["USERNAME"]
password = os.environ["PASSWORD"]
opcua_rest_url = os.environ["OPC_UA_REST_URL"]
opcua_server_url = os.environ["OPC_UA_SERVER_URL"]
model_index_url = os.environ["MODEL_INDEX_URL"]
ory_url = os.environ["ORY_URL"]

In [ ]:
# Getting ory bearer token
auth_client = AUTH_CLIENT(rest_url=ory_url, username=username, password=password)
auth_client.request_new_ory_token()

### Download data from modelindex api

In [ ]:
# Connecting to ModelIndex APIs 
model_data = ModelIndex(url=model_index_url)

In [ ]:
# Listed sites on the model index api server
namespaces = model_data.get_namespace_array()
namespaces

In [ ]:
# Types of Objects
object_types_json = model_data.get_object_types()
object_types = AnalyticsHelper(object_types_json)
object_types.dataframe

In [ ]:
# Unique types of Objects
object_types_unique = object_types.dataframe[["Id", "Name"]].drop_duplicates()
object_types_unique

In [ ]:
# To get typeId by type name of an object
object_type_id = model_data.get_object_type_id_from_name("SiteType")
object_type_id

In [ ]:
str(object_type_id.split(":")[1])

In [ ]:
# To get the objects of a type
sites_json = model_data.get_objects_of_type("SiteType")

# Send the returned JSON into a normalizer to get Id, Type, Name, Props and Vars as columns
sites = AnalyticsHelper(sites_json)
sites.list_of_names()

In [ ]:
# Analytics helper
sites.variables_as_dataframe()

In [ ]:
sites.list_of_ids()

In [ ]:
# Selecting the single site
site_id = sites.list_of_ids()[0]
site_id

In [ ]:
# Get all stringsets for one park
string_sets_for_first_park_as_json = model_data.get_object_descendants(
    "StringSetType", [site_id], "PV_Assets"
)
string_sets_for_first_park = AnalyticsHelper(string_sets_for_first_park_as_json)
string_sets_for_first_park.dataframe

In [ ]:
# Ancestors of an object type, get all trackers that are ancestor of the parks string sets

trackers_as_json = model_data.get_object_ancestors(
    "TrackerType", string_sets_for_first_park.list_of_ids(), "PV_Serves"
)
trackers = AnalyticsHelper(trackers_as_json)
trackers.variables_as_dataframe()

### Download data from the opc ua api

In [ ]:
namespace_list = object_types.namespaces_as_list(namespaces)

# Initating the OPC UA API with a fixed namespace list
opc_data = OPC_UA(rest_url=opcua_rest_url, opcua_url=opcua_server_url, namespaces=namespace_list)

### Read Historical Events

In [ ]:
from typing import List
import pandas as pd
import copy
import json
from requests import HTTPError
import math
import requests

In [ ]:
def get_historical_raw_values(opc_data,
    start_time,
    end_time,
    variable_list,
    limit_start_index=None,
    limit_num_records=None,
) -> pd.DataFrame:
    
    # Create a new variable list to remove pydantic models
    vars = opc_data._get_variable_list_as_list(variable_list)

    extended_variables = []
    for var in vars:
        extended_variables.append(
            {
                "NodeId": var,
            }
        )
    body = copy.deepcopy(opc_data.body)
    body["StartTime"] = start_time.strftime("%Y-%m-%dT%H:%M:%SZ")
    body["EndTime"] = end_time.strftime("%Y-%m-%dT%H:%M:%SZ")
    body["ReadValueIds"] = extended_variables
    if limit_start_index is not None and limit_num_records is not None:
        body["Limit"] = {
            "StartIndex": limit_start_index,
            "NumRecords": limit_num_records
        }
    
    print(body)

    # Try making the request, if fails check if it is due to ory client
    content = request_from_api(
        rest_url=opcua_rest_url,
        method="POST",
        endpoint="values/historical",
        data=json.dumps(body, default=opc_data.json_serial),
        headers=opc_data.headers,
        extended_timeout=True,
    )

    df_result = pd.json_normalize(content, record_path=['HistoryReadResults', 'DataValues'], meta=[['HistoryReadResults', 'NodeId', 'IdType'], ['HistoryReadResults', 'NodeId','Id'],['HistoryReadResults', 'NodeId','Namespace']] )
    df_result.rename(
            columns={
                "Value.Type": "ValueType",
                "Value.Body": "Value",
                "SourceTimestamp": "Timestamp",
                "HistoryReadResults.NodeId.IdType": "IdType",
                "HistoryReadResults.NodeId.Id": "Id",
                "HistoryReadResults.NodeId.Namespace": "Namespace",
            },
            errors="raise",
            inplace=True,
        )

    return df_result

In [ ]:
start_time=(datetime.datetime.now() - datetime.timedelta(30))
end_time=(datetime.datetime.now() - datetime.timedelta(29))
variable_list=trackers.variables_as_list(["AngleSetpoint"])

In [ ]:
get_historical_raw_values(opc_data,
    start_time,
    end_time,
    variable_list,
    limit_start_index=None,
    limit_num_records=None,
)

In [ ]:
base_event_type = "0:0:2782"

In [ ]:
def get_event_types(opc_data, base_event_type: str) -> pd.DataFrame:

    namespace, id_type, id = map(int, base_event_type.split(':'))

    body = copy.deepcopy(opc_data.body)
    body["BaseEventType"] = {
        "Id": str(id),
        "Namespace": namespace,
        "IdType": id_type
    }
    print(body)

    try:
        # Try making the request, if fails check if it is due to ory client
        content = request_from_api(
            rest_url=opcua_rest_url,
            method="POST",
            endpoint="events/types",
            data=json.dumps(body, default=opc_data.json_serial),
            headers=opc_data.headers,
            extended_timeout=True,
        )

    except HTTPError as e:
        if opc_data.auth_client is not None:
            opc_data.check_auth_client(json.loads(e.response.content))
        else:
            raise RuntimeError(f'Error message {e}')
    
    df_result = pd.DataFrame(content['EventTypes'])

    df_result['BrowseName'] = df_result['BrowseName'].apply(lambda x: x.get('Name', None))
    df_result['Id'] = df_result['NodeId'].apply(lambda x: x.get('Id', None))
    df_result['Namespace'] = df_result['NodeId'].apply(lambda x: x.get('Namespace'))

    df_result['Namespace'] = df_result['Namespace'].fillna(0).astype(int)
    df_result.drop(columns=['NodeId', 'DisplayName'], inplace=True)
    

    return df_result

In [ ]:
# df_result_flat = pd.json_normalize(df_result['BrowseName'])
    # df_result_flat.columns = ['BrowseName.' + str(col) for col in df_result_flat.columns]

    # df_result_node = pd.json_normalize(df_result['NodeId'])
    # df_result_node.columns = ['NodeId.' + str(col) for col in df_result_node.columns]

    # df_result = pd.concat([df_result, df_result_flat, df_result_node], axis=1)
    
    # df_result.rename(
    #         columns={
    #             "BrowseName.Name": "BrowseName",
    #             "DisplayName": "TypeName",
    #             "NodeId.Id": "TypeId",
    #             "NodeId.Namespace": "Namespace",
    #         },
    #         errors="raise",
    #         inplace=True,
    #     )

#df_result['TypeName'] = df_result['BrowseName'].apply(lambda x: x.get('Name') if isinstance(x, dict) else np.nan)
#df_result['TypeId'] = df_result['NodeId'].apply(lambda x: x.get('Id') if isinstance(x, dict) else np.nan)
#df_result = df_result.drop(columns=['BrowseName', 'NodeId'])
#df_result = pd.json_normalize(content, record_path=['EventTypes'], meta=[['BrowseName', 'Name'], ['NodeId', 'Id']])
# df_result.columns = df_result.columns.map(lambda x: x.split(".")[-1])

In [ ]:
def get_event_type_id_from_name(df_result, type_name: str) -> str:
        """Get event type id and namespace from type name

        Args:
            type_name (str): type name

        Returns:
            str: an object of event type id and namespace in the form of a tuple
        """
        event_type = df_result[df_result["BrowseName"] == type_name]
        if not event_type.empty:
            event_type_id, namespace = event_type[["Id", "Namespace"]].values[0]
        else:
            event_type_id = None
        
        event_type_id = f"{namespace}:0:{event_type_id}"

        return event_type_id

In [ ]:
type_id = get_event_type_id_from_name(df_result, "SiteEventType")

In [ ]:
int(type_id.split(":")[1])

In [ ]:
type_id[1]

In [ ]:
df_result = get_event_types(opc_data, base_event_type)

In [ ]:
df_result

In [ ]:
#print(df_result["EventTypes"])#[0]

In [ ]:
#test_etypes= df_result["EventTypes"][0]

In [ ]:
#test_etypes

In [ ]:
#pd.json_normalize(df_result['EventTypes'].explode().reset_index(drop=True))

In [ ]:
#df = pd.DataFrame(test_etypes)
#df_expanded = pd.json_normalize(df.explode().reset_index(drop=True))

In [ ]:
#df_expanded = pd.json_normalize(test_etypes)

In [ ]:
#df_expanded

In [ ]:
def read_historical_events(opc_data,
    start_time,
    end_time,
    fields_list,
    event_type_noded_id,
    object_type_id,
) -> pd.DataFrame:
    
    body = copy.deepcopy(opc_data.body)
    body["StartTime"] = start_time.strftime("%Y-%m-%dT%H:%M:%SZ")
    body["EndTime"] = end_time.strftime("%Y-%m-%dT%H:%M:%SZ")
    body["Fields"] = fields_list
    body["WhereClause"] = {
            "EventTypeNodedId": {
                "Id": int(event_type_noded_id.split(":")[2]),
                "Namespace": int(event_type_noded_id.split(":")[0]),
                "IdType": int(event_type_noded_id.split(":")[1])
            }
        }
    body["ReadValueIds"] = [
            {
                "NodeId": {
                    "Id": int(object_type_id.split(":")[2]),
                    "Namespace": int(object_type_id.split(":")[0]),
                    "IdType": int(object_type_id.split(":")[1])
                }
            }
        ]
    
    print(body)

    # Try making the request, if fails check if it is due to ory client
    content = request_from_api(
        rest_url=opcua_rest_url,
        method="POST",
        endpoint="events/read",
        data=json.dumps(body, default=opc_data.json_serial),
        headers=opc_data.headers,
        extended_timeout=True,
    )

    df_result = pd.json_normalize(content)

    return df_result

In [ ]:
start_time=(datetime.datetime.now() - datetime.timedelta(30))
end_time=(datetime.datetime.now() - datetime.timedelta(29))
#variable_list=trackers.variables_as_list(["AngleMeasured", "AngleSetpoint"])

In [ ]:
event_type_noded_id = get_event_type_id_from_name(df_result, "EnergyAndPowerMeterEventType")

In [ ]:
object_type_id = model_data.get_object_type_id_from_name("EnergyAndPowerMeterEventType")

In [ ]:
object_type_id

In [ ]:
fields_list = ["Time", "EventId", "EventType", "SourceName", "Message", "Severity", "Quality"]

In [ ]:
df_hist_event = read_historical_events(opc_data,
    start_time,
    end_time,
    fields_list,
    event_type_noded_id,
    object_type_id,
)

In [ ]:
df_hist_event

In [ ]:
df_hist_event["EventsResult"][0]

### End

In [ ]:
# Live value data of trackers
live_value = opc_data.get_values(
    trackers.variables_as_list(["AngleMeasured", "AngleSetpoint"])
)
live_value

In [ ]:
# Raw historic value data of trackers
one_day_raw_historic_tracker_data = opc_data.get_historical_raw_values(
    start_time=(datetime.datetime.now() - datetime.timedelta(30)),
    end_time=(datetime.datetime.now() - datetime.timedelta(29)),
    variable_list=trackers.variables_as_list(["AngleSetpoint"]),
)
one_day_raw_historic_tracker_data

In [ ]:
# Aggregated historic value data of trackers, 1 days worth of data 30 days ago
one_day_historic_tracker_data = opc_data.get_historical_aggregated_values(
    start_time=(datetime.datetime.now() - datetime.timedelta(30)),
    end_time=(datetime.datetime.now() - datetime.timedelta(29)),
    pro_interval=3600000,
    agg_name="Average",
    variable_list=trackers.variables_as_list(["AngleMeasured"]),
)
one_day_historic_tracker_data